In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import sys
sys.path.append('/content/drive/MyDrive/Transformer-BasedGPTModel')

In [48]:
import torch
from GPTLanguageModel import GPTLanguageModel
import os

In [49]:
batch_size = 8
block_size = 256 # maximum number of tokens the model can use as input at once when making predictions.
max_iters = 25000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 5
n_layer = 6
dropout = 0.2
torch.manual_seed(1337)

In [50]:
with open('/content/drive/MyDrive/Transformer-BasedGPTModel/vefxistyaosani.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [51]:
!pip install dill
import dill as pickle

In [52]:
with open('/content/drive/MyDrive/Transformer-BasedGPTModel/tokenizer', 'rb') as f:
    tokenizer = pickle.load(f)

In [38]:
data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
torch.save(data, '/content/drive/MyDrive/Transformer-BasedGPTModel/data.pt')
#data = torch.load('/content/drive/MyDrive/Transformer-BasedGPTModel/data.pt', weights_only=True)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [53]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [54]:
x, y = get_batch('train')
print('inputs:')
print(x.shape)
print('targets:')
print(y.shape)

inputs:
torch.Size([8, 256])
targets:
torch.Size([8, 256])


In [55]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [56]:
model = GPTLanguageModel()
print(model)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

GPTLanguageModel(
  (token_embedding_table): Embedding(67634, 32)
  (position_embedding_table): Embedding(256, 32)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-1): 2 x Head(
            (key): Linear(in_features=32, out_features=16, bias=False)
            (query): Linear(in_features=32, out_features=16, bias=False)
            (value): Linear(in_features=32, out_features=16, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=32, out_features=32, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=32, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=32, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
   

In [57]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [58]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 11.1233, val loss 11.1245
step 500: train loss 3.3250, val loss 3.3243
step 1000: train loss 2.8359, val loss 2.8478
step 1500: train loss 2.6470, val loss 2.6670
step 2000: train loss 2.5354, val loss 2.5579
step 2500: train loss 2.4440, val loss 2.4677
step 3000: train loss 2.3687, val loss 2.4029
step 3500: train loss 2.3047, val loss 2.3401
step 4000: train loss 2.2476, val loss 2.2853
step 4500: train loss 2.1969, val loss 2.2360
step 5000: train loss 2.1398, val loss 2.1855
step 5500: train loss 2.1005, val loss 2.1417
step 6000: train loss 2.0606, val loss 2.1038
step 6500: train loss 2.0320, val loss 2.0739
step 7000: train loss 2.0061, val loss 2.0546
step 7500: train loss 1.9797, val loss 2.0188
step 8000: train loss 1.9639, val loss 2.0006
step 8500: train loss 1.9544, val loss 1.9993
step 9000: train loss 1.9364, val loss 1.9840
step 9500: train loss 1.9250, val loss 1.9706
step 10000: train loss 1.9104, val loss 1.9535
step 10500: train loss 1.9090, val 

In [63]:
with open('/content/drive/MyDrive/Transformer-BasedGPTModel/GPTModel', 'wb') as f:
    pickle.dump(model, f)

with open('/content/drive/MyDrive/Transformer-BasedGPTModel/GPTModel', 'rb') as f:
    gpt_model = pickle.load(f)

In [64]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(tokenizer.decode(gpt_model.generate(context, max_new_tokens=200)[0].tolist()))


და ამარგომა!”
ღრეჯი ნელი შეორნიტა
აწვიდე, შესწრალი ვერა უმუნდომართა, ვაწვრ გეხდა მონარი,
და ბედი აღმან ფა სიდოფობესა მიგი ჭირით მიტკინდეს დადიდით;
ზხინასა ნაახვის, კვლა მიჰკვდეს, მის ხვდეს ლხითა-მარ
ფრომეს ცე


In [ ]:
"""
მიძერებუნ მისი მატანი დავ-და ვარდი.

ჩემცად მათი ჩემნსა მბავისათლა, ჰტყვილე, მოვეტომცა საყულისა.
მოშო, ვარ დაჯობი, კაშინისა რა სიკულებო ჟამათა.
მეფეს სახლი, უმძმომენე, შენუ სადგების ვერთგან სგვახოსა საბარულგულობა;
და ასა შესწრვი ვაცა ვარგთალო, ღმერთაცა მიენ,
ჩემნ მის გულ ვაუწერათ ვითა შემოარულთა ულხერებს!


“რაცვის ქვეცობ ამოატკაოს, მომგცე ერთითქცა აქებელულსა”.
კაცითა ჭიანობულნი ჩვრსა გშუმოდგონეს, და ღეთი იქმართ.
იაყამეს და და გმამისისსნა, შეარ ვკლა კვლა ესროსტყალთა.
და მძარნებლა: “გათქვარე რაცა გითხვეთ, ჩემი ღაბად,
და აწ აეცოდორი დავდიჭი ფირფ ხმობა კლურალვად”.
სანურო კაცრაზებს, მოაშინესა, ლოქითად ჭამჭვნით ნაუწქება;
და ლაშქმნით ნა მეფრი ოსიან, მოგვსგულსა, ბევრა შეცნისხდით”.

რაცა ყმა ელთულსაგარ არ ჰკრთობა, მონით, ლგურთ, ბრძელი ებარი ყმა და მჭანითითა;
ვითა მქი ომაწყევის მიხარულს, საწუშარებს,
და აუბოვე ძლეთა უმჭობენნ განდა დასით იგი კბიარნ!”

“შე, დაღავჰშუსვეს დაუნდა, დახანანმცა შენია,
მათგან უცნობა აჰყვნეთითა ჭმ[UNK:30228]ქემათქვენანითანის.
წათ მის”.
და, ათუთრთა ე, პყველელთა და ჰმჯაპნდა მცრზემან პირი ტუონდესია;
თუმათ ალისაგი ხდა გაჭავშმშანესმანე უნ მზედით;
და უბრძანნა, მოდნად ჩემსა რაშინ ზომგად მონალთა ცხამე,
ფიმწუკიცნა მზართ -ნანდურებელად,
და ესწმავენ და თმოცა შენთვარ ჩემგონესფადკუბით,

და მასას ბაღლსა მცნი ვნახერდლია წვილოოს, მინად მოხითა რაუბობა,
მოიშორით მიედიფდო, გლურდეტითა ატევნებდალ, ისჯა,
მინ მისი მჭირისობა მოეკიდარვის, მან, ჩვენ უცრემლისა საქმსაზისა,
და ერთ უნდამსავი იკირვად საგხარე, უმოვისნეს ფრარისა:
და ავინახავსა, რა ნა ზეტარასა, ფრიდომსა თუ შასამიხო შოგიესა!”

“ნავსა ლახ საღა ქალი მოეტულისა, ვახვთა უსლებურ ნენი ბარძლისა”;
რუცა ვარეტ ყოვლასა მტისა ტყვისა, მივთოს,
ვცოჩნნიე ცუდად ათგაგნ მუნა მჭვრომილისა.
და ესე მე გამამე, ბრძანანის ცედს მძმალსა ბოურფთა ანსურისა?
და შეპოვესა მიგთან, ჩვენთა შეფერდე-გწუნა, გან ყმანარა;
და თუ ჩემ-დძქვს შენ დაშებადვეს ასადავან,
კვლა მხარ იტის: “იღასხეს, მაგრა ღიმი მოთვილი მთვინა,
და მათნ აჰკვივ ვერ დიბილთა, თველთა უგვითხვედ თამათათა.

“მაშთა საუგობ, ამის ამბადეს, არისა ესწორად,
ვითა საოფთაღილა, მატითვა უთხრაფად გამალთა,
მოვმაკობნესლა, მე მისაჭვრეტსა, მისცა, გარდ გაეშან.
და მოდე თქვა: “ჩემი მოალატანს!
"""